In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/home/kenn/anaconda3/envs/tf_base/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 一列，300行
x_data = np.linspace(-1,1,300)[:,np.newaxis]
# 噪声
noise = np.random.normal(0,0.05,x_data.shape)
# y
y_data = np.square(x_data) - 0.5 + noise

In [3]:
def add_layer(inputs, in_size,out_size, n_layer, activation_function=None):
    layer_name = 'layer%s' % n_layer
    # 把所有内容可视化的时候包含在layer节点内
    with tf.name_scope("layer"):
        with tf.name_scope("weights"):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name="W") # 权重矩阵 行=输入维度，列=输出维度（下一层输入维度）
            # 把参数数据总结到直方图中
            tf.summary.histogram(layer_name+"/weights", Weights)
        with tf.name_scope("biases"):
            biases = tf.Variable(tf.zeros([1,out_size]) + 0.1, name="b")           # 偏置向量 行=1，列=输出维度
            tf.summary.histogram(layer_name+"/biases", biases)
        with tf.name_scope("Wx_plus_b"):
            Wx_plus_b = tf.matmul(inputs, Weights) + biases              # 其实是xW，如果行列定义反过来，这里可以Wx，xW的比较符合数据储存直觉
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        tf.summary.histogram(layer_name+"/outputs", outputs)
        return outputs

#输入神经元：1个，隐藏10个，输出神经元1个
#输入层: None行,1列。None表示这个维度不固定，此即样本数
#placeholder里面需要定义dtype，否则报错
#name是可视化的时候显示的节点名称
#name_scope("inputs")把2个input合并为一个输入节点
with tf.name_scope("inputs"):
    xs = tf.placeholder(tf.float32, [None,1], name = "x_input")
    ys = tf.placeholder(tf.float32, [None,1], name = "y_input")
#第一层（隐藏层1）：输入1，输出10，用relu
l1 = add_layer(xs, 1, 10, n_layer=1, activation_function=tf.nn.sigmoid)
#第二层（输出层）：输入为上一层输出=10，输出1,
prediction = add_layer(l1, 10, 1, n_layer=2)

#均方误差
with tf.name_scope("loss"):
    loss =  tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction, 
                                                   name="square"),reduction_indices=[1],
                                         name="sum"),
                           name="mean")
    tf.summary.scalar("loss",loss)
    
with tf.name_scope("train"):
    train_step = tf. train.GradientDescentOptimizer(0.1).minimize(loss)
    
sess = tf.Session()
# 将上述可视化合并
merged = tf.summary.merge_all()
# 将以上结构写入文件
writer = tf.summary.FileWriter("logs/simple_NN",sess.graph)
# 终端激活对应环境后执行 tensorboard --logdi'file:///home/kenn/tensorflow_learning/logs'
# localhost:6006看
init = tf.global_variables_initializer()
sess.run(init)
for i in range(10000):
    #通过placeholder定义输入的话，都要用feed_dict载入数据
    sess.run(train_step, feed_dict = {xs:x_data, ys:y_data})
    if i%50 == 0:
        result = sess.run(merged, feed_dict = {xs:x_data, ys:y_data})
        writer.add_summary(result,i)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.scatter(x_data,y_data)

In [ ]:
%matplotlib
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.scatter(x_data,y_data)
plt.ion()
plt.show()
sess.run(init)
for i in range(10000):
    #通过placeholder定义输入的话，都要用feed_dict载入数据
    sess.run(train_step, feed_dict = {xs:x_data, ys:y_data})
    if i%50 == 0:
        #print(sess.run(loss,feed_dict = {xs:x_data, ys:y_data}))
        try:
            ax.lines.remove(lines[0])
        except Exception:
            pass
        prediction_value = sess.run(prediction, feed_dict = {xs:x_data, ys:y_data})
        lines = ax.plot(x_data, prediction_value,'r-',lw=5)
        plt.pause(0.1)